In [22]:
import json
import random
import spacy
from sklearn.metrics import precision_recall_fscore_support
from collections import defaultdict

# Carregar o modelo treinado
model_output_dir = "../../model/spaCy_model/"
nlp = spacy.load(model_output_dir)

# Carregar o conjunto de validação
validation_data_path = "../../data/processed/validation_data.json"
with open(validation_data_path, "r") as file:
    validation_data = json.load(file)

# Preparar as previsões e os rótulos para avaliação
true_entities = []
pred_entities = []

for text, annotations in validation_data:
    # Criar o objeto Doc
    doc = nlp(text)
    
    # Adicionar as entidades reais e previstas
    true_ents = [(start, end, label) for start, end, label in annotations["entities"]]
    pred_ents = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
    
    # Adicionar apenas se o número de entidades coincidir
    if len(true_ents) == len(pred_ents):
        true_entities.append(true_ents)
        pred_entities.append(pred_ents)

# Função para converter as entidades em um formato de lista plana para avaliação
def flatten_entities(entities_list):
    flattened_labels = []
    flattened_offsets = []
    for entities in entities_list:
        for start, end, label in entities:
            flattened_labels.append(label)
            flattened_offsets.append((start, end))
    return flattened_labels, flattened_offsets

# Converter as listas de entidades
true_labels, true_offsets = flatten_entities(true_entities)
pred_labels, pred_offsets = flatten_entities(pred_entities)

# Calcular as métricas de precisão, recall e f-score manualmente
precision, recall, fscore, _ = precision_recall_fscore_support(true_labels, pred_labels, average="weighted", zero_division="warn")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F-Score: {fscore:.2f}")

# Contar acertos e erros por tipo de entidade
correct_counts = defaultdict(int)
missed_counts = defaultdict(int)

# Comparação direta entre entidades verdadeiras e previstas
for true, pred in zip(true_entities, pred_entities):
    true_set = set(true)
    pred_set = set(pred)
    
    # Verifica acertos e erros
    for entity in true_set:
        if entity in pred_set:
            correct_counts[entity[2]] += 1  # entity[2] é o nome da entidade
        else:
            missed_counts[entity[2]] += 1

# Resumo dos acertos e erros por tipo de entidade
print("\nResumo dos Acertos e Erros por Tipo de Entidade:")
for label in set(true_labels):
    print(f"{label}: Correto = {correct_counts[label]}, Perdido = {missed_counts[label]}")

# # Visualização de Amostras
# num_samples = 5
# samples = random.sample(validation_data, num_samples)

# print("\nAmostras de Avaliação do Modelo no Conjunto de Validação:\n")
# for i, (text, annotations) in enumerate(samples, 1):
#     doc = nlp(text)
#     true_entities_sample = [(start, end, label) for start, end, label in annotations["entities"]]
#     pred_entities_sample = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]

#     print(f"Amostra {i}:")
#     print(f"Texto: {text}\n")
#     print(f"Entidades Verdadeiras: {true_entities_sample}")
#     print(f"Entidades Previstas: {pred_entities_sample}\n")
#     print("-" * 50)

# Calcular métricas manualmente para cada tipo de entidade
for label in correct_counts:
    total_true = correct_counts[label] + missed_counts[label]  # Total de entidades verdadeiras do tipo
    total_pred = correct_counts[label]  # Total de entidades previstas corretamente
    
    # Calcula precisão, recall e F1-score manualmente
    precision_manual = correct_counts[label] / total_pred if total_pred > 0 else 0
    recall_manual = correct_counts[label] / total_true if total_true > 0 else 0
    f1_manual = 2 * (precision_manual * recall_manual) / (precision_manual + recall_manual) if (precision_manual + recall_manual) > 0 else 0
    
    print(f"\nTipo de Entidade: {label}")
    print(f"Precisão Manual: {precision_manual:.2f}")
    print(f"Recall Manual: {recall_manual:.2f}")
    print(f"F1-Score Manual: {f1_manual:.2f}")
    print("-" * 50)



Precisão: 1.00
Recall: 1.00
F-Score: 1.00

Resumo dos Acertos e Erros por Tipo de Entidade:
PRODUCT_NAME: Correto = 464, Perdido = 0
CLUSTER: Correto = 463, Perdido = 1
BRAND: Correto = 462, Perdido = 2
CATEGORY: Correto = 1507, Perdido = 16
PRICE: Correto = 463, Perdido = 0

Tipo de Entidade: CATEGORY
Precisão Manual: 1.00
Recall Manual: 0.99
F1-Score Manual: 0.99
--------------------------------------------------

Tipo de Entidade: CLUSTER
Precisão Manual: 1.00
Recall Manual: 1.00
F1-Score Manual: 1.00
--------------------------------------------------

Tipo de Entidade: PRODUCT_NAME
Precisão Manual: 1.00
Recall Manual: 1.00
F1-Score Manual: 1.00
--------------------------------------------------

Tipo de Entidade: PRICE
Precisão Manual: 1.00
Recall Manual: 1.00
F1-Score Manual: 1.00
--------------------------------------------------

Tipo de Entidade: BRAND
Precisão Manual: 1.00
Recall Manual: 1.00
F1-Score Manual: 1.00
--------------------------------------------------
